In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import copy

import random
from itertools import chain, combinations, tee
import time

plt.rcParams['text.usetex'] = True

# Functions

In [2]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]

def equals(a, b, tol = 1E-3):
    if abs(a-b) <= tol:
        return True
    else:
        return False
    
def equals_array(arr_1, arr_2, tol = 1E-2):
    if np.linalg.norm(arr_1 - arr_2) <= tol:
        return True
    else:
        return False
    

In [3]:
def latency_max(flow_max, coeff):
    
    assert np.all(coeff >= 0.0), "coeff should be non-negative"
    assert len(coeff.shape) == 1, "coeff should be a 1-D array."
    assert coeff.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    
    return coeff[0] + max(coeff[1] * (flow_max - coeff[2]), 0)

In [4]:
# arr_1 = np.array([[1, 2, 3], [4, 5, 6]])
# np.linalg.norm(arr_1)

# Download Groups, Routes to Edges Data:

In [5]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
# df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_3.csv')

# df_od_flow_data
# df_data

In [6]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])

# Test git

In [7]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 612.5396169,
  'Demand (eligible group, 1)': 60.02888245620001,
  'VoT (eligible group, 1)': 0.04180157639979069,
  'Demand (eligible group, 2)': 36.752377014000004,
  'VoT (eligible group, 2)': 0.1494057158119658,
  'Demand (ineligible group, 1)': 93.71856138570001,
  'VoT (ineligible group, 1)': 0.29949880593262945,
  'Demand (ineligible group, 2)': 134.75871571800002,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 287.28108032610004,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 98.03491986,
  'Demand (eligible group, 1)': 9.607422146280001,
  'VoT (eligible group, 1)': 0.04180157639979069,
  'Demand (eligible group, 2)': 5.8820951916,
  'VoT (eligible group, 2)': 0.149

In [8]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [9]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

edge_to_od_dict = {}
num_edges = int(np.max(od_to_edges_array)) + 1
# print("num_edges:", num_edges)

for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]
    
# od_to_edges_array
# edge_to_od_dict

In [10]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array_base = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array_base[od_index, group_index] = od_value[VoT_name]

print(demand_array)
# VoT_array_base

[[ 60.02888246  36.75237701  93.71856139 134.75871572 287.28108033]
 [  9.60742215   5.88209519  14.99934274  21.56768237  45.97837741]
 [111.52479072  68.28048412 174.11523449 250.36177509 533.72578417]
 [ 25.07588823  15.35258463  39.14909081  56.29281031 120.00603653]
 [ 95.06622324  58.20381015 148.41971587 213.41397053 454.95978264]
 [ 23.76644387  14.550884    37.10475421  53.35324134 113.73940995]
 [150.04450532  91.86398285 234.25315626 336.83460377 718.07013259]
 [ 25.30172796  35.37078297  62.99613901  74.8724603   59.63978735]
 [  5.68890543   7.95285759  14.16421351  16.83451606  13.40956279]
 [ 21.56665392  30.14930191  53.6965669   63.81969017  50.83568424]
 [  5.39189555   7.5376499   13.42471953  15.95560927  12.70946808]
 [ 34.04016665  47.58676358  84.75306798 100.73110539  80.23753567]
 [ 10.40251464  13.91885761  23.5888008   37.36114412  61.2429735 ]
 [ 39.43629223  52.76686989  89.42595844 141.63738759 232.1742275 ]
 [  9.85945787  13.19223236  22.35736222  35.410

In [11]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'

T = 5
VoT_array = np.zeros((VoT_array_base.shape[0], VoT_array_base.shape[1], T))

for t in range(T):
    df_perturbation_data = pd.read_csv(directory_path + 'perturbations_1_' + str(t) + '.csv')
    perturbation_array = df_perturbation_data.to_numpy()[:, 1:]
    VoT_array[:, :, t] = VoT_array_base * perturbation_array
    
# VoT_array_base
# perturbation_array

# Download Latency Parameters Data

In [12]:
directory_path_latency = '../data/pems_latency_inference___101_N_Sep_to_Nov_2024/'
df_latency_params = pd.read_csv(directory_path_latency + 'latency_params.csv')

# list(df_latency_params.loc[:, "Palo Alto"])

In [13]:
dict_latency_params = {}

city_list = list(df_latency_params.columns)[1:]

for city in city_list:
#     if city != "Belmont":
    if 1 == 1:
        dict_latency_params[city] = {}
        dict_latency_params[city]["Flow (at bend)"] = df_latency_params.loc[:, city][0]
        dict_latency_params[city]["Latency (at bend)"] = df_latency_params.loc[:, city][1]
        dict_latency_params[city]["Slope (after bend)"] = df_latency_params.loc[:, city][2]

dict_latency_params

{'Palo Alto': {'Flow (at bend)': np.float64(861.9885),
  'Latency (at bend)': np.float64(1.326448252),
  'Slope (after bend)': np.float64(0.000782666)},
 'East Palo Alto': {'Flow (at bend)': np.float64(1001.517857),
  'Latency (at bend)': np.float64(2.213126553),
  'Slope (after bend)': np.float64(0.000584484)},
 'Redwood City': {'Flow (at bend)': np.float64(881.1846667),
  'Latency (at bend)': np.float64(4.892192375),
  'Slope (after bend)': np.float64(0.001563724)},
 'Belmont': {'Flow (at bend)': np.float64(1278.948125),
  'Latency (at bend)': np.float64(1.199911179),
  'Slope (after bend)': np.float64(0.001994138)},
 'San Mateo': {'Flow (at bend)': np.float64(1034.092826),
  'Latency (at bend)': np.float64(5.541006284),
  'Slope (after bend)': np.float64(0.002147262)},
 'Burlingame': {'Flow (at bend)': np.float64(845.15),
  'Latency (at bend)': np.float64(1.503111345),
  'Slope (after bend)': np.float64(0.000306601)},
 'Millbrae': {'Flow (at bend)': np.float64(853.1818182),
  'Laten

In [14]:
# num_edges = 7
num_gp_lanes = 3

num_el = 3
num_groups = demand_array.shape[1]

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]
    

In [15]:
## Set lambdas:

lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

## Initialize tau, alpha values:

filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

# directory_inits = '../data/opt_values___2_el_groups/'
directory_inits = '../data/opt_CBCP_values___' + str(num_el) + '_el_groups/'
df_inits = pd.read_csv(directory_inits + filename_segment + '___tau_B_stats_CBCP.csv')

print("filename_segment:", filename_segment)
print()

inits_tau_arr_as_object = df_inits.to_numpy()[:, 1:6]
inits_B_arr_as_object = df_inits.to_numpy()[0, 7]

argmin_tau = np.zeros((num_edges, T))
argmin_B = 0

for e in range(num_edges):
    for t in range(T):
        argmin_tau[e, t] = inits_tau_arr_as_object[e, t]
        argmin_B = inits_B_arr_as_object

print("argmin_tau:\n", argmin_tau)
print()
print("argmin_B:\n", argmin_B)


filename_segment: 1_1_1

argmin_tau:
 [[0.92 0.28 0.46 0.48 0.93]
 [0.95 0.32 0.79 0.58 0.71]
 [0.52 2.24 2.07 1.35 2.2 ]
 [1.26 0.86 1.71 1.38 2.46]
 [2.13 2.29 2.26 0.59 1.31]
 [0.53 0.44 0.69 0.51 0.38]
 [0.78 0.26 0.48 0.36 0.74]]

argmin_B:
 7.02


In [16]:
# argmin_tau = np.array([[0.0, 0.3194, 0.3194, 0.0, 0.0], \
#                        [0.2498, 0.2498, 0.2498, 0.0, 0.0], \
#                        [0.0, 0.0, 0.0, 0.9995, 0.9995], \
#                        [0.0, 1.0281, 1.0281, 1.0281, 0.0], \
#                        [1.6043, 0.0, 0.0, 1.6043, 0.0], \
#                        [0.0, 0.1922, 0.1922, 0.1922, 0.0], \
#                        [0.0, 0.0, 0.0, 0.2178, 0.2178]])

# argmin_B = 10.6925

# Compute CBCP and DBCP comparison statistics

In [17]:
# demand_array
# edge_to_od_dict
# od_to_edges_array

# demand_array[:, 0:3]
# demand_array[:, 3:5]
# demand_array

# np.sum(demand_array[:, 0:3], axis=1).shape

In [18]:
demand_array_el = np.sum(demand_array[:, 0:3], axis=1)
demand_array_in = np.sum(demand_array[:, 3:5], axis=1)

demand_array_el_across_edges = np.zeros(num_edges)
demand_array_in_across_edges = np.zeros(num_edges)
demand_array_across_edges = np.zeros(num_edges)

for e in range(num_edges):
    demand_array_el_across_edges[e] = np.sum([demand_array_el[od] for od in edge_to_od_dict[e]])
    demand_array_in_across_edges[e] = np.sum([demand_array_in[od] for od in edge_to_od_dict[e]])
    demand_array_across_edges[e] = demand_array_el_across_edges[e] + demand_array_in_across_edges[e]

# demand_array_el_across_edges[e]

In [19]:
print("demand_array_el_across_edges:", demand_array_el_across_edges)
print("demand_array_across_edges:", demand_array_across_edges)

array([1507.76022969, 1766.88182121, 2298.02265606, 1990.28188165,
       2635.79268911, 2188.02527641, 2341.16466508])

array([4848.10363245, 5174.23084722, 6793.71793021, 6047.63829921,
       7944.86026391, 6645.07072329, 7113.41109785])

In [21]:
# Read data from csv and store into numpy files

directory_CBCP = '../data/opt_CBCP_values___' + str(num_el) + '_el_groups/'
directory_DBCP = '../data/opt_DBCP_values___' + str(num_el) + '_el_groups/'

tau_CBCP_dict = {}
tau_time_averaged_CBCP_dict = {}
B_CBCP_dict = {}
percent_express_lane_use_CBCP_dict = {}
avg_travel_time_CBCP_dict = {}
total_costs_CBCP_dict = {}

tau_DBCP_dict = {}
tau_time_averaged_DBCP_dict = {}
alpha_DBCP_dict = {}
percent_express_lane_use_DBCP_dict = {}
avg_travel_time_DBCP_dict = {}
total_costs_DBCP_dict = {}


# lambdas_array = np.array([[1.0, 1.0, 1.0], \
#                           [1.0, 5.0, 1.0], \
#                           [1.0, 10.0, 1.0], \
#                           [5.0, 5.0, 1.0], \
#                           [5.0, 10.0, 1.0], \
#                           [10.0, 10.0, 1.0], \
#                           [1.0, 1.0, 0.0], \
#                           [1.0, 5.0, 0.0], \
#                           [5.0, 10.0, 0.0]])

lambdas_array = np.array([[1.0, 1.0, 1.0], \
                          [1.0, 5.0, 1.0], \
                          [1.0, 10.0, 1.0], \
                          [5.0, 5.0, 1.0], \
                          [5.0, 10.0, 1.0], \
                          [10.0, 10.0, 1.0], \
                          [1.0, 1.0, 0.0], \
                          [1.0, 5.0, 0.0], \
                          [5.0, 10.0, 0.0], \
                          [5.0, 1.0, 1.0], \
                          [10.0, 5.0, 1.0], \
                          [10.0, 1.0, 1.0], \
                          [20.0, 1.0, 1.0], \
                          [5.0, 1.0, 0.0], \
                          [10.0, 5.0, 0.0], \
                          [10.0, 1.0, 0.0], \
                          [20.0, 1.0, 0.0], \
                          [5.0, 0.0, 1.0], \
                          [10.0, 0.0, 1.0], \
                          [20.0, 0.0, 1.0], \
                         ])

for lambdas_index in range(lambdas_array.shape[0]):
    lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
    
    filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))
    
    lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
    lambdas = (lambda_E, lambda_R, lambda_I)
    filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

    # Inputting CBCP data:

    df_CBCP = pd.read_csv(directory_CBCP + filename_segment + '___tau_B_stats_CBCP.csv')
    tau_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 1:6].astype(float)
    tau_time_averaged_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 6].astype(float)
    B_CBCP_dict[lambdas] = df_CBCP.to_numpy()[0, 7]
    percent_express_lane_use_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 8:11].astype(float)
    avg_travel_time_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 11:13].astype(float)
    total_costs_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 13:].astype(float)

    # Inputting DBCP data:

    df_DBCP = pd.read_csv(directory_DBCP + filename_segment + '___tau_alpha_stats_DBCP.csv')
    tau_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 1:6].astype(float)
    tau_time_averaged_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 6].astype(float)
    alpha_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 7:12].astype(float)
    percent_express_lane_use_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 12:15].astype(float)
    avg_travel_time_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 15:17].astype(float)
    total_costs_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 17:].astype(float)
    


In [22]:
 total_costs_DBCP_dict[(5.0, 10.0, 1.0)]

array([[  2320.02,  39112.43,   1071.15,  40001.01],
       [  4104.09,  56840.99,    744.25,  69918.97],
       [ 13757.59, 192087.93,   4936.34, 211512.51],
       [  3184.95,  46671.28,   2758.9 ,  35007.07],
       [ 18988.85, 272315.52,   5465.08, 312609.01],
       [  3783.39,  53049.32,   1346.92,  58497.05],
       [  6187.57,  85250.21,   1585.39, 100334.19]])

In [23]:
# Process data into average or total costs, and storing into dict_avg_stats:

dict_avg_stats = {}

for lambdas_index in range(lambdas_array.shape[0]):
    lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
    lambdas = (lambda_E, lambda_R, lambda_I)

    # Processing CBCP data:
    
    dict_index = (lambda_E, lambda_R, lambda_I, 'CBCP')
    
    dict_avg_stats[dict_index] = {}
    
    dict_avg_stats[dict_index]['percent of overall'] \
        = np.sum(percent_express_lane_use_CBCP_dict[lambdas][:, 0] * demand_array_across_edges) \
            / np.sum(demand_array_across_edges)
    dict_avg_stats[dict_index]['percent of eligible'] \
        = np.sum(percent_express_lane_use_CBCP_dict[lambdas][:, 1] * demand_array_el_across_edges) \
            / np.sum(demand_array_el_across_edges)
    dict_avg_stats[dict_index]['percent of ineligible'] \
        = np.sum(percent_express_lane_use_CBCP_dict[lambdas][:, 2] * demand_array_in_across_edges) \
            / np.sum(demand_array_in_across_edges)
    
    dict_avg_stats[dict_index]['average travel time, express'] \
        = np.sum(avg_travel_time_CBCP_dict[lambdas][:, 0])
    dict_avg_stats[dict_index]['average travel time, general purpose'] \
        = np.sum(avg_travel_time_CBCP_dict[lambdas][:, 1])
    
    dict_avg_stats[dict_index]['total travel cost, eligible'] \
        = np.sum(total_costs_CBCP_dict[lambdas][:, 0])
    dict_avg_stats[dict_index]['total travel cost, ineligible'] \
        = np.sum(total_costs_CBCP_dict[lambdas][:, 1])
    dict_avg_stats[dict_index]['total toll revenue'] \
        = np.sum(total_costs_CBCP_dict[lambdas][:, 2])
    dict_avg_stats[dict_index]['total societal cost'] \
        = np.sum(total_costs_CBCP_dict[lambdas][:, 3])

#     # Processing DBCP data:
    
    dict_index = (lambda_E, lambda_R, lambda_I, 'DBCP')
    
    dict_avg_stats[dict_index] = {}
    
    dict_avg_stats[dict_index]['percent of overall'] \
        = np.sum(percent_express_lane_use_DBCP_dict[lambdas][:, 0] * demand_array_across_edges) \
            / np.sum(demand_array_across_edges)
    dict_avg_stats[dict_index]['percent of eligible'] \
        = np.sum(percent_express_lane_use_DBCP_dict[lambdas][:, 1] * demand_array_el_across_edges) \
            / np.sum(demand_array_el_across_edges)
    dict_avg_stats[dict_index]['percent of ineligible'] \
        = np.sum(percent_express_lane_use_DBCP_dict[lambdas][:, 1] * demand_array_in_across_edges) \
            / np.sum(demand_array_in_across_edges)
    
    dict_avg_stats[dict_index]['average travel time, express'] \
        = np.sum(avg_travel_time_DBCP_dict[lambdas][:, 0])
    dict_avg_stats[dict_index]['average travel time, general purpose'] \
        = np.sum(avg_travel_time_DBCP_dict[lambdas][:, 1])
    
    dict_avg_stats[dict_index]['total travel cost, eligible'] \
        = np.sum(total_costs_DBCP_dict[lambdas][:, 0])
    dict_avg_stats[dict_index]['total travel cost, ineligible'] \
        = np.sum(total_costs_DBCP_dict[lambdas][:, 1])
    dict_avg_stats[dict_index]['total toll revenue'] \
        = np.sum(total_costs_DBCP_dict[lambdas][:, 2])
    dict_avg_stats[dict_index]['total societal cost'] \
        = np.sum(total_costs_DBCP_dict[lambdas][:, 3])
    

In [24]:
# lambdas = (1.0, 1.0, 1.0)

# percent_express_lane_use_CBCP_dict[lambdas][:, 0]

# np.sum(percent_express_lane_use_CBCP_dict[lambdas][:, 0] * demand_array_across_edges) \
#             / np.sum(demand_array_across_edges)

# dict_avg_stats[(1.0, 1.0, 1.0, 'CBCP')]

In [25]:
avg_stats_array = np.zeros((lambdas_array.shape[0] * 2, 13))

for lambdas_index in range(lambdas_array.shape[0]):
    lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
    
    # Storing CBCP avg stats as array:
    
    dict_index = (lambda_E, lambda_R, lambda_I, 'CBCP')
    
    avg_stats_array[2 * lambdas_index, 0] = lambda_E
    avg_stats_array[2 * lambdas_index, 1] = lambda_R
    avg_stats_array[2 * lambdas_index, 2] = lambda_I
    
    # Here, "0" indicates CBCP
    avg_stats_array[2 * lambdas_index, 3] = 0
    
    avg_stats_array[2 * lambdas_index, 4] = dict_avg_stats[dict_index]['percent of overall']
    avg_stats_array[2 * lambdas_index, 5] = dict_avg_stats[dict_index]['percent of eligible']
    avg_stats_array[2 * lambdas_index, 6] = dict_avg_stats[dict_index]['percent of ineligible']
    
    avg_stats_array[2 * lambdas_index, 7] = dict_avg_stats[dict_index]['average travel time, express']
    avg_stats_array[2 * lambdas_index, 8] = dict_avg_stats[dict_index]['average travel time, general purpose']

    avg_stats_array[2 * lambdas_index, 9] = dict_avg_stats[dict_index]['total travel cost, eligible']
    avg_stats_array[2 * lambdas_index, 10] = dict_avg_stats[dict_index]['total travel cost, ineligible']
    avg_stats_array[2 * lambdas_index, 11] = dict_avg_stats[dict_index]['total toll revenue']
    avg_stats_array[2 * lambdas_index, 12] = dict_avg_stats[dict_index]['total societal cost']
    
    # Storing DBCP avg stats as array:
    
    dict_index = (lambda_E, lambda_R, lambda_I, 'DBCP')
    
    avg_stats_array[2 * lambdas_index + 1, 0] = lambda_E
    avg_stats_array[2 * lambdas_index + 1, 1] = lambda_R
    avg_stats_array[2 * lambdas_index + 1, 2] = lambda_I
    
    # Here, "1" indicates DBCP
    avg_stats_array[2 * lambdas_index + 1, 3] = 1
    
    avg_stats_array[2 * lambdas_index + 1, 4] = dict_avg_stats[dict_index]['percent of overall']
    avg_stats_array[2 * lambdas_index + 1, 5] = dict_avg_stats[dict_index]['percent of eligible']
    avg_stats_array[2 * lambdas_index + 1, 6] = dict_avg_stats[dict_index]['percent of ineligible']
    
    avg_stats_array[2 * lambdas_index + 1, 7] = dict_avg_stats[dict_index]['average travel time, express']
    avg_stats_array[2 * lambdas_index + 1, 8] = dict_avg_stats[dict_index]['average travel time, general purpose']

    avg_stats_array[2 * lambdas_index + 1, 9] = dict_avg_stats[dict_index]['total travel cost, eligible']
    avg_stats_array[2 * lambdas_index + 1, 10] = dict_avg_stats[dict_index]['total travel cost, ineligible']
    avg_stats_array[2 * lambdas_index + 1, 11] = dict_avg_stats[dict_index]['total toll revenue']
    avg_stats_array[2 * lambdas_index + 1, 12] = dict_avg_stats[dict_index]['total societal cost']


In [26]:
avg_stats_array.shape

(40, 13)

In [27]:
column_names = []
column_names += ["lambda_E", "lambda_R", "lambda_I"]
column_names += ["CBCP (=0) or DBCP (=1)"]
column_names += ["% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

df_avg_stats_to_save = pd.DataFrame(avg_stats_array, index=None, columns=column_names)

df_avg_stats_to_save

,lambda_E,lambda_R,lambda_I,CBCP (=0) or DBCP (=1),% overall users using express lanes,% eligible users using express lanes,% ineligible users using express lanes,Average travel time (express lanes),Average travel time (general purpose lanes),Total travel cost (eligible users),Total travel cost (ineligible users),Total toll revenue,Total societal cost
0,1.0,1.0,1.0,0.0,17.083434,32.917250,9.268677,21.43,24.97,51331.16,756870.65,14114.06,794087.74
1,1.0,1.0,1.0,1.0,21.702749,32.977559,32.626013,22.77,24.20,52196.98,735077.09,13095.12,774178.94
2,1.0,5.0,1.0,0.0,16.607750,35.368219,7.346107,21.37,25.14,51413.06,761992.08,10689.72,759956.62
3,1.0,5.0,1.0,1.0,20.012656,32.675598,32.719961,21.93,24.53,52439.05,745435.27,17033.12,712708.76
4,1.0,10.0,1.0,0.0,16.863381,24.903230,12.899190,21.14,25.08,52085.48,759966.21,22649.36,585557.96
5,1.0,10.0,1.0,1.0,17.593434,16.112556,16.178109,21.21,24.90,53210.00,755186.49,29118.51,517211.30
6,5.0,5.0,1.0,0.0,17.943420,35.266640,9.394225,21.46,25.08,51678.70,760340.48,14483.39,946317.01
7,5.0,5.0,1.0,1.0,19.886501,22.752711,22.718433,21.84,24.58,52809.87,746377.45,22963.50,895609.23
8,5.0,10.0,1.0,0.0,14.411511,34.021531,4.731341,21.30,25.50,51990.02,771777.40,9666.29,935064.57
9,5.0,10.0,1.0,1.0,19.982895,29.457675,29.518945,21.92,24.50,52326.46,745327.68,17908.03,827879.81


In [28]:
directory_to_save = "../data/stats_compare___" + str(num_el) + "_el_groups/"
# filename = "opt_CBCP_params___" + random_string + '.csv'

filename = 'avg_stats___' + str(num_el) + '_el_groups.csv'

df_avg_stats_to_save.to_csv(directory_to_save + filename)

In [37]:
# Stats array are: DBCP minus CBCP

stats_compare_percentage_array = np.zeros((lambdas_array.shape[0], 11))

for lambdas_index in range(lambdas_array.shape[0]):
# for lambdas_index in [12]:
    print()
    print("lambdas_index:", lambdas_index)
    print("avg_stats_array[2 * lambdas_index, 4:12]:\n", avg_stats_array[2 * lambdas_index, 4:12])
    
    stats_compare_percentage_array[lambdas_index, 0:3] = lambdas_array[lambdas_index, 0:3]
    
    stats_compare_percentage_array[lambdas_index, 3:] = \
        100 * (avg_stats_array[2 * lambdas_index + 1, 4:12] - avg_stats_array[2 * lambdas_index, 4:12]) \
            / avg_stats_array[2 * lambdas_index, 4:12]




lambdas_index: 0
avg_stats_array[2 * lambdas_index, 4:12]:
 [1.70834336e+01 3.29172496e+01 9.26867690e+00 2.14300000e+01
 2.49700000e+01 5.13311600e+04 7.56870650e+05 1.41140600e+04]

lambdas_index: 1
avg_stats_array[2 * lambdas_index, 4:12]:
 [1.66077502e+01 3.53682190e+01 7.34610691e+00 2.13700000e+01
 2.51400000e+01 5.14130600e+04 7.61992080e+05 1.06897200e+04]

lambdas_index: 2
avg_stats_array[2 * lambdas_index, 4:12]:
 [1.68633810e+01 2.49032299e+01 1.28991902e+01 2.11400000e+01
 2.50800000e+01 5.20854800e+04 7.59966210e+05 2.26493600e+04]

lambdas_index: 3
avg_stats_array[2 * lambdas_index, 4:12]:
 [1.79434196e+01 3.52666404e+01 9.39422475e+00 2.14600000e+01
 2.50800000e+01 5.16787000e+04 7.60340480e+05 1.44833900e+04]

lambdas_index: 4
avg_stats_array[2 * lambdas_index, 4:12]:
 [1.44115109e+01 3.40215309e+01 4.73134079e+00 2.13000000e+01
 2.55000000e+01 5.19900200e+04 7.71777400e+05 9.66629000e+03]

lambdas_index: 5
avg_stats_array[2 * lambdas_index, 4:12]:
 [1.59386673e+01 3.5

In [38]:
# # stats_compare_percentage_array
# lambdas_array

In [39]:
column_names = []
column_names += ["lambda_E", "lambda_R", "lambda_I"]
column_names += ["% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue"]

df_stats_compare_to_save = pd.DataFrame(stats_compare_percentage_array, index=None, columns=column_names)

df_stats_compare_to_save

,lambda_E,lambda_R,lambda_I,% overall users using express lanes,% eligible users using express lanes,% ineligible users using express lanes,Average travel time (express lanes),Average travel time (general purpose lanes),Total travel cost (eligible users),Total travel cost (ineligible users),Total toll revenue
0,1.0,1.0,1.0,27.039736,0.183217,252.002920,6.252916,-3.083700,1.686734,-2.879430,-7.219326
1,1.0,5.0,1.0,20.501908,-7.613109,345.405452,2.620496,-2.426412,1.995582,-2.172832,59.341124
2,1.0,10.0,1.0,4.329218,-35.299334,25.419569,0.331126,-0.717703,2.158989,-0.628938,28.562176
3,5.0,5.0,1.0,10.828934,-35.483758,141.834041,1.770736,-1.993620,2.188851,-1.836418,58.550588
4,5.0,10.0,1.0,38.659265,-13.414611,523.902315,2.910798,-3.921569,0.647124,-3.427118,85.262702
5,10.0,10.0,1.0,5.908984,-25.607094,321.348298,0.143472,-0.950495,3.306938,-0.834603,66.904158
6,1.0,1.0,0.0,18.317430,-36.956987,1978.442744,2.678571,-2.844725,2.426919,-2.715719,361.517063
7,1.0,5.0,0.0,13.181038,-21.779183,301.664338,0.430005,-1.755755,2.297915,-1.563430,71.041210
8,5.0,10.0,0.0,3.001377,-19.255087,291.860016,0.762268,-0.317586,3.895272,-0.317437,46.290534
9,5.0,1.0,1.0,52.972685,12.957024,764.553467,9.654851,-4.429370,1.122810,-4.113734,-53.311606


In [40]:
# directory_to_save = "../data/stats_compare/"
# filename = "opt_CBCP_params___" + random_string + '.csv'

filename = 'compare_stats___' + str(num_el) + '_el_groups.csv'

df_stats_compare_to_save.to_csv(directory_to_save + filename)

## <font color='red'>STOP. End of Code to run.</font> 

# End of code to run.

In [ ]:
# np.sum(np.array([1, 2, 3]) * np.array([5, 3, 1]))


In [ ]:
# avg_travel_time_CBCP_dict[(1.0, 1.0, 1.0)]

In [ ]:
# tau_CBCP_dict = {}
# tau_time_averaged_CBCP_dict = {}
# B_CBCP_dict = {}
# percent_express_lane_use_CBCP_dict = {}
# avg_travel_time_CBCP_dict = {}
# total_costs_CBCP_dict = {}

# tau_DBCP_dict = {}
# tau_time_averaged_DBCP_dict = {}
# alpha_DBCP_dict = {}
# percent_express_lane_use_DBCP_dict = {}
# avg_travel_time_DBCP_dict = {}
# total_costs_DBCP_dict = {}


# directory_CBCP = '../data/opt_CBCP_values___' + str(num_el) + '_el_groups/'
# directory_DBCP = '../data/opt_DBCP_values___' + str(num_el) + '_el_groups/'

# lambdas_index = 4
# lambda_E, lambda_R, lambda_I = lambdas_array[lambdas_index]
# lambdas = (lambda_E, lambda_R, lambda_I)
# filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

# # Inputting CBCP data:

# df_CBCP = pd.read_csv(directory_CBCP + filename_segment + '___tau_B_stats_CBCP.csv')
# tau_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 1:6]
# tau_time_averaged_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 6]
# B_CBCP_dict[lambdas] = df_CBCP.to_numpy()[0, 7]
# percent_express_lane_use_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 8:11]
# avg_travel_time_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 11:12]
# total_costs_CBCP_dict[lambdas] = df_CBCP.to_numpy()[:, 12:]

# # Inputting DBCP data:

# df_DBCP = pd.read_csv(directory_DBCP + filename_segment + '___tau_alpha_stats_DBCP.csv')
# tau_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 1:6]
# tau_time_averaged_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 6]
# alpha_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 7:12]
# percent_express_lane_use_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 12:15]
# avg_travel_time_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 15:16]
# total_costs_DBCP_dict[lambdas] = df_DBCP.to_numpy()[:, 16:]



## Test:

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## Linear Approximation for Latency Function:

In [ ]:
# Variables:
v = cp.Variable(1)
            
# Objective:
func = v - 1 + cp.square(cp.maximum(v-1, 0))
objective = cp.Minimize(func)

# Constraints:
constraints = [-3.0 <= v, v <= 3.0]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("v.value:", v.value)
